In [8]:
# 1 导入包
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import normalize
from keras.layers import LSTM
from keras.layers import Embedding, SimpleRNN, Bidirectional
from keras.preprocessing.sequence import TimeseriesGenerator



In [9]:
# 2 读取数据集

# dataframe = pd.read_csv("total.csv",header=None).values # 取消第一行作为表头
train_set = pd.read_csv("software_train_set.csv")
test_set = pd.read_csv("software_test_set.csv")



# 3 读标签
# label = dataframe.loc[:,['label']]
y_train = train_set['label']
y_test = test_set['label']

# 标签转化为one-hot编码 
y_train = keras.utils.to_categorical(y_train, num_classes=23)
y_test = keras.utils.to_categorical(y_test, num_classes=23)

print("y_train.shape",y_train.shape)

y_train.shape (1508190, 23)


In [10]:
# 
########## 一般是转为浮点数向量

# train_set = np.array(train_set[1:,:]).astype(np.float32) # 去掉第一行字段 
# test_set = np.array(test_set[1:,:]).astype(np.float32)



In [11]:
# 读数据
train_data = train_set.iloc[:,1:]
test_data = test_set.iloc[:,1:]


# 5 特征归一化  用MLP的时候用，LSTM不用，用embedding层 ,但是这里有2列数值太大了，
min_max_scaler = sklearn.preprocessing.MinMaxScaler(feature_range=(0, 65535))

train_data[['tcp.options.timestamp.tsval','tcp.options.timestamp.tsecr']] = min_max_scaler.fit_transform(train_data[['tcp.options.timestamp.tsval','tcp.options.timestamp.tsecr']])
test_data[['tcp.options.timestamp.tsval','tcp.options.timestamp.tsecr']] = min_max_scaler.fit_transform(test_data[['tcp.options.timestamp.tsval','tcp.options.timestamp.tsecr']])

train_data = train_data.values
test_data = test_data.values

train_set_GA = np.array(train_data).astype(np.int64) # 转为整数向量
test_set_GA = np.array(test_data).astype(np.int64)



In [12]:
# 数据集生成器
# train_label_ = np.insert(y_train, 0, 0, axis=0)
# test_label_ = np.insert(y_test, 0, 0, axis=0)

In [13]:
# 利用TimesereisGenerator生成序列数据
time_steps = 1
batch_size = 1024

train_generator = TimeseriesGenerator(train_set_GA, y_train, length=time_steps, sampling_rate=1, batch_size=batch_size)
test_generator = TimeseriesGenerator(test_set_GA, y_test, length=time_steps, sampling_rate=1, batch_size=batch_size)

In [14]:

########## Embedding 层输入整数向量

# train_set = np.array(train_set).astype(np.int32) # 转成整数
# x_test_set = np.array(x_test_set).astype(np.int32)


print(train_set_GA)
print(type(test_set_GA))

[[ 74  60   1 ...   0  18   0]
 [ 58  44   0 ...   0   2   0]
 [ 54  40   1 ...   0   0   0]
 ...
 [260 246   1 ...   0  10   3]
 [ 74  60   0 ...   0  17   0]
 [ 74  60   1 ...   0  18   0]]
<class 'numpy.ndarray'>


In [15]:
##################  LSTM
max_features = 65536  # 作为特征的(单词)索引个数
# x_train = x_train[:, None]
model = Sequential()
# model.add(Dense(64, activation='relu', input_dim=104))
# model.add(Dropout(0.5))

# keras.layers.embeddings.Embedding(input_dim, output_dim, 
#                                   init='uniform', input_length=None, W_regularizer=None, activity_regularizer=None, W_constraint=None, mask_zero=False, weights=None, dropout=0.0)
model.add(Embedding(max_features, 32)) # 接收二维整数向量，输出三维向量
# print(model.layers[0].get_weights()[0].shape)

# exit()
# model.add(Dropout(0.5))
# model.add(Dense(32, activation='relu'))
# model.add(LSTM(32)) # 接收三维向量
model.add(Bidirectional(LSTM(32)))
model.add(Dense(23, activation='softmax')) 
print(model.summary())

print(model.get_layer(index=0).output_shape)
# loss = [binary_crossentropy,categorical_crossentropy]
# optimizer = [rmsprop,] RMSProp对比Adamgrad增加了指数平滑
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['acc'])

model.fit(train_set_GA, y_train , epochs=5, batch_size=69, validation_data=[test_set_GA, y_test], validation_freq=1)
# score = model.evaluate(test_set_GA, y_test, batch_size=128)

#################


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          2097152   
                                                                 
 bidirectional (Bidirectiona  (None, 64)               16640     
 l)                                                              
                                                                 
 dense (Dense)               (None, 23)                1495      
                                                                 
Total params: 2,115,287
Trainable params: 2,115,287
Non-trainable params: 0
_________________________________________________________________
None
(None, None, 32)
Epoch 1/5
 3860/21858 [====>.........................] - ETA: 3:22 - loss: 1.3350 - acc: 0.5302

In [ ]:
history = model.fit_generator(train_generator, epochs=250, verbose=2, steps_per_epoch=25,
                                validation_data=test_generator, shuffle=0)

In [ ]:
model.save("Embedding1.h5")

In [ ]:

score = model.evaluate(test_set_GA, y_test, batch_size=1)
